In [9]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

import itertools
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_colwidth = 10000

In [10]:
# read provided dataset
df = pd.read_csv("../receipt_data.csv")
df.head(1)

,comments,currency,lastUpdated,purchaseTime,status,uploadedTime,totalConfidence,lastOpenAt,lastVerifiedAt,taxTotal,total,deletedAt,retailerName,retailerCountry,retailerVatNumber,rawData,receiptImage
0,NaN,GBP,2019-06-15 17:00:57,2019-06-14 18:20:00,REVIEWED,2019-06-14 22:11:40,0.7,NaN,2019-06-15 17:00:57,NaN,NaN,NaN,Selfridges,GB,NaN,"{""message"":""SUCCESS: Result available"",""status"":""done"",""status_code"":3,""token"":null,""success"":true,""code"":202,""duplicate"":false,""duplicateToken"":null,""result"":{""establishment"":""Selfridges"",""validatedEstablishment"":false,""date"":""2019-06-14 18:20:00"",""total"":30.000,""url"":""www.selfridges.com"",""phoneNumber"":""8569 4004156"",""paymentMethod"":""VISA"",""address"":""400 Oxford Street London W1A 1AB"",""validatedTotal"":false,""subTotal"":30.000,""validatedSubTotal"":true,""cash"":""0.000"",""change"":""0.000"",""tax"":0.000,""discount"":0.000,""subTotalConfidence"":0.99,""totalConfidence"":0.7,""cashConfidence"":0.0,""changeConfidence"":0.0,""roundingConfidence"":0.0,""otherData"":[],""lineItems"":[{""qty"":0,""desc"":""FULTON BROLLYMA41568569 400415685692"",""unit"":"""",""price"":0.000,""descClean"":""You 193 were served by : Ines FULTON BROLLYMA 400415685692"",""lineTotal"":18.000,""productCode"":""41568569""},{""qty"":0,""desc"":""JJ DOT HYDR - TRA 82401412"",""unit"":"""",""price"":0.000,""descClean"":""JJ DOT HYDR - TRA"",""lineTotal"":12.000,""productCode"":""82401412""}],""summaryItems"":[{""qty"":2,""desc"":""Total 2 Items"",""unit"":"""",""price"":0.000,""descClean"":""Total Items"",""lineTotal"":30.000,""productCode"":""""},{""qty"":0,""desc"":""Visa"",""unit"":"""",""price"":0.000,""descClean"":""Visa"",""lineTotal"":30.000,""productCode"":""""},{""qty"":14,""desc"":""009999 Date 14 / 06 / 19"",""unit"":"""",""price"":0.000,""descClean"":""Date / 06 / 19"",""lineTotal"":18.200,""productCode"":""009999""}]}}",https://s3-eu-west-1.amazonaws.com/wevat-receipts/5955cc32-894b-4dbd-8c1f-f54fbd95d8e8


In [11]:
# filter retailerName to be used as label and rawData as features
df = df.filter(['retailerName', 'rawData'])
df.head(5)

,retailerName,rawData
0,Selfridges,"{""message"":""SUCCESS: Result available"",""status"":""done"",""status_code"":3,""token"":null,""success"":true,""code"":202,""duplicate"":false,""duplicateToken"":null,""result"":{""establishment"":""Selfridges"",""validatedEstablishment"":false,""date"":""2019-06-14 18:20:00"",""total"":30.000,""url"":""www.selfridges.com"",""phoneNumber"":""8569 4004156"",""paymentMethod"":""VISA"",""address"":""400 Oxford Street London W1A 1AB"",""validatedTotal"":false,""subTotal"":30.000,""validatedSubTotal"":true,""cash"":""0.000"",""change"":""0.000"",""tax"":0.000,""discount"":0.000,""subTotalConfidence"":0.99,""totalConfidence"":0.7,""cashConfidence"":0.0,""changeConfidence"":0.0,""roundingConfidence"":0.0,""otherData"":[],""lineItems"":[{""qty"":0,""desc"":""FULTON BROLLYMA41568569 400415685692"",""unit"":"""",""price"":0.000,""descClean"":""You 193 were served by : Ines FULTON BROLLYMA 400415685692"",""lineTotal"":18.000,""productCode"":""41568569""},{""qty"":0,""desc"":""JJ DOT HYDR - TRA 82401412"",""unit"":"""",""price"":0.000,""descClean"":""JJ DOT HYDR - TRA"",""lineTotal"":12.000,""productCode"":""82401412""}],""summaryItems"":[{""qty"":2,""desc"":""Total 2 Items"",""unit"":"""",""price"":0.000,""descClean"":""Total Items"",""lineTotal"":30.000,""productCode"":""""},{""qty"":0,""desc"":""Visa"",""unit"":"""",""price"":0.000,""descClean"":""Visa"",""lineTotal"":30.000,""productCode"":""""},{""qty"":14,""desc"":""009999 Date 14 / 06 / 19"",""unit"":"""",""price"":0.000,""descClean"":""Date / 06 / 19"",""lineTotal"":18.200,""productCode"":""009999""}]}}"
1,Harrods,"{""message"":""SUCCESS: Result available"",""status"":""done"",""status_code"":3,""token"":null,""success"":true,""code"":202,""duplicate"":false,""duplicateToken"":null,""result"":{""establishment"":""Harrods"",""validatedEstablishment"":false,""date"":""2019-06-19 13:24:00"",""total"":86.000,""url"":""harrods.com"",""phoneNumber"":""020 7730 1234"",""paymentMethod"":"""",""address"":""LONDON S1 ot Road"",""validatedTotal"":false,""subTotal"":86.000,""validatedSubTotal"":true,""cash"":""0.000"",""change"":""0.000"",""tax"":0.000,""discount"":0.000,""subTotalConfidence"":0.6,""totalConfidence"":0.0,""cashConfidence"":0.0,""changeConfidence"":0.0,""roundingConfidence"":0.0,""otherData"":[],""lineItems"":[{""qty"":0,""desc"":""Rewards 0005708615 Saving FRAGRANCE"",""unit"":"""",""price"":0.000,""descClean"":""Rewards Saving FRAGRANCE f8.60"",""lineTotal"":86.000,""productCode"":""0005708615""}],""summaryItems"":[{""qty"":0,""desc"":""Sub Total"",""unit"":"""",""price"":0.000,""descClean"":""Sub Total"",""lineTotal"":86.000,""productCode"":""""},{""qty"":1,""desc"":""TOTAL ( 1 item )"",""unit"":"""",""price"":0.000,""descClean"":""TOTAL ( item )"",""lineTotal"":77.400,""productCode"":""""},{""qty"":0,""desc"":""Chinese Mobile Tendered"",""unit"":"""",""price"":0.000,""descClean"":""Chinese Mobile Tendered"",""lineTotal"":77.400,""productCode"":""""},{""qty"":0,""desc"":""Rewards Points balance"",""unit"":"""",""price"":0.000,""descClean"":""Rewards Points balance"",""lineTotal"":194.800,""productCode"":""""}]}}"
2,Boots,"{""message"":""SUCCESS: Result available"",""status"":""done"",""status_code"":3,""token"":null,""success"":true,""code"":202,""duplicate"":false,""duplicateToken"":null,""result"":{""establishment"":""Boots"",""validatedEstablishment"":false,""date"":""2019-06-18 18:24:00"",""total"":20.460,""url"":"""",""phoneNumber"":""(0115) 9410199"",""paymentMethod"":"""",""address"":"""",""validatedTotal"":true,""subTotal"":0.000,""validatedSubTotal"":false,""cash"":""50.500"",""change"":""30.040"",""tax"":0.000,""discount"":0.000,""subTotalConfidence"":0.0,""totalConfidence"":0.99,""cashConfidence"":0.99,""changeConfidence"":0.99,""roundingConfidence"":0.0,""otherData"":[""f"",""Boots UK Limited"",""NOTTINGHAM - 6"",""18/06/2019"",""(0115) 9410199"",""18:24"",""Served by: LAURIEN GOOD"",""1404"",""619 7646 0006 144 *6330******

In [12]:

# we can see that some Boot tags are duplicated
df['retailerName'].value_counts()

Selfridges           1961
Boots                1601
Harrods              1188
Holland & Barrett     582
John Lewis            458
TK Maxx               433
Debenhams             394
Boots UK Limited      383
Name: retailerName, dtype: int64

In [13]:
# clean some data with bad labels
df.retailerName = df.retailerName.str.replace('Boots.*', 'Boots', regex=True)

# Boot retailer tags must be now deduplicated
# The counts should be closer for the model be more balanced
df['retailerName'].value_counts()

Boots                1984
Selfridges           1961
Harrods              1188
Holland & Barrett     582
John Lewis            458
TK Maxx               433
Debenhams             394
Name: retailerName, dtype: int64

In [14]:
ignore = df[df.rawData.apply(lambda x: not isinstance(x, str))]
ignore.head(5)

,retailerName,rawData
685,Boots,NaN
1326,Boots,NaN
1537,John Lewis,NaN
1872,Holland & Barrett,NaN
3050,Boots,NaN


In [15]:
# drop rows with empty or not string rawData 
dfc = df[df.rawData.apply(lambda x: isinstance(x, str))]
dfc.head(5)

,retailerName,rawData
0,Selfridges,"{""message"":""SUCCESS: Result available"",""status"":""done"",""status_code"":3,""token"":null,""success"":true,""code"":202,""duplicate"":false,""duplicateToken"":null,""result"":{""establishment"":""Selfridges"",""validatedEstablishment"":false,""date"":""2019-06-14 18:20:00"",""total"":30.000,""url"":""www.selfridges.com"",""phoneNumber"":""8569 4004156"",""paymentMethod"":""VISA"",""address"":""400 Oxford Street London W1A 1AB"",""validatedTotal"":false,""subTotal"":30.000,""validatedSubTotal"":true,""cash"":""0.000"",""change"":""0.000"",""tax"":0.000,""discount"":0.000,""subTotalConfidence"":0.99,""totalConfidence"":0.7,""cashConfidence"":0.0,""changeConfidence"":0.0,""roundingConfidence"":0.0,""otherData"":[],""lineItems"":[{""qty"":0,""desc"":""FULTON BROLLYMA41568569 400415685692"",""unit"":"""",""price"":0.000,""descClean"":""You 193 were served by : Ines FULTON BROLLYMA 400415685692"",""lineTotal"":18.000,""productCode"":""41568569""},{""qty"":0,""desc"":""JJ DOT HYDR - TRA 82401412"",""unit"":"""",""price"":0.000,""descClean"":""JJ DOT HYDR - TRA"",""lineTotal"":12.000,""productCode"":""82401412""}],""summaryItems"":[{""qty"":2,""desc"":""Total 2 Items"",""unit"":"""",""price"":0.000,""descClean"":""Total Items"",""lineTotal"":30.000,""productCode"":""""},{""qty"":0,""desc"":""Visa"",""unit"":"""",""price"":0.000,""descClean"":""Visa"",""lineTotal"":30.000,""productCode"":""""},{""qty"":14,""desc"":""009999 Date 14 / 06 / 19"",""unit"":"""",""price"":0.000,""descClean"":""Date / 06 / 19"",""lineTotal"":18.200,""productCode"":""009999""}]}}"
1,Harrods,"{""message"":""SUCCESS: Result available"",""status"":""done"",""status_code"":3,""token"":null,""success"":true,""code"":202,""duplicate"":false,""duplicateToken"":null,""result"":{""establishment"":""Harrods"",""validatedEstablishment"":false,""date"":""2019-06-19 13:24:00"",""total"":86.000,""url"":""harrods.com"",""phoneNumber"":""020 7730 1234"",""paymentMethod"":"""",""address"":""LONDON S1 ot Road"",""validatedTotal"":false,""subTotal"":86.000,""validatedSubTotal"":true,""cash"":""0.000"",""change"":""0.000"",""tax"":0.000,""discount"":0.000,""subTotalConfidence"":0.6,""totalConfidence"":0.0,""cashConfidence"":0.0,""changeConfidence"":0.0,""roundingConfidence"":0.0,""otherData"":[],""lineItems"":[{""qty"":0,""desc"":""Rewards 0005708615 Saving FRAGRANCE"",""unit"":"""",""price"":0.000,""descClean"":""Rewards Saving FRAGRANCE f8.60"",""lineTotal"":86.000,""productCode"":""0005708615""}],""summaryItems"":[{""qty"":0,""desc"":""Sub Total"",""unit"":"""",""price"":0.000,""descClean"":""Sub Total"",""lineTotal"":86.000,""productCode"":""""},{""qty"":1,""desc"":""TOTAL ( 1 item )"",""unit"":"""",""price"":0.000,""descClean"":""TOTAL ( item )"",""lineTotal"":77.400,""productCode"":""""},{""qty"":0,""desc"":""Chinese Mobile Tendered"",""unit"":"""",""price"":0.000,""descClean"":""Chinese Mobile Tendered"",""lineTotal"":77.400,""productCode"":""""},{""qty"":0,""desc"":""Rewards Points balance"",""unit"":"""",""price"":0.000,""descClean"":""Rewards Points balance"",""lineTotal"":194.800,""productCode"":""""}]}}"
2,Boots,"{""message"":""SUCCESS: Result available"",""status"":""done"",""status_code"":3,""token"":null,""success"":true,""code"":202,""duplicate"":false,""duplicateToken"":null,""result"":{""establishment"":""Boots"",""validatedEstablishment"":false,""date"":""2019-06-18 18:24:00"",""total"":20.460,""url"":"""",""phoneNumber"":""(0115) 9410199"",""paymentMethod"":"""",""address"":"""",""validatedTotal"":true,""subTotal"":0.000,""validatedSubTotal"":false,""cash"":""50.500"",""change"":""30.040"",""tax"":0.000,""discount"":0.000,""subTotalConfidence"":0.0,""totalConfidence"":0.99,""cashConfidence"":0.99,""changeConfidence"":0.99,""roundingConfidence"":0.0,""otherData"":[""f"",""Boots UK Limited"",""NOTTINGHAM - 6"",""18/06/2019"",""(0115) 9410199"",""18:24"",""Served by: LAURIEN GOOD"",""1404"",""619 7646 0006 144 *6330******

In [16]:
rawData = dfc.rawData
parsedRawData = rawData.apply(json.loads).apply(pd.Series)
parsedRawData.head(2)

,message,status,status_code,token,success,code,duplicate,duplicateToken,result
0,SUCCESS: Result available,done,3,None,True,202,False,None,"{'establishment': 'Selfridges', 'validatedEstablishment': False, 'date': '2019-06-14 18:20:00', 'total': 30.0, 'url': 'www.selfridges.com', 'phoneNumber': '8569 4004156', 'paymentMethod': 'VISA', 'address': '400 Oxford Street London W1A 1AB', 'validatedTotal': False, 'subTotal': 30.0, 'validatedSubTotal': True, 'cash': '0.000', 'change': '0.000', 'tax': 0.0, 'discount': 0.0, 'subTotalConfidence': 0.99, 'totalConfidence': 0.7, 'cashConfidence': 0.0, 'changeConfidence': 0.0, 'roundingConfidence': 0.0, 'otherData': [], 'lineItems': [{'qty': 0, 'desc': 'FULTON BROLLYMA41568569 400415685692', 'unit': '', 'price': 0.0, 'descClean': 'You 193 were served by : Ines FULTON BROLLYMA 400415685692', 'lineTotal': 18.0, 'productCode': '41568569'}, {'qty': 0, 'desc': 'JJ DOT HYDR - TRA 82401412', 'unit': '', 'price': 0.0, 'descClean': 'JJ DOT HYDR - TRA', 'lineTotal': 12.0, 'productCode': '82401412'}], 'summaryItems': [{'qty': 2, 'desc': 'Total 2 Items', 'unit': '', 'price': 0.0, 'descClean': 'Total Items', 'lineTotal': 30.0, 'productCode': ''}, {'qty': 0, 'desc': 'Visa', 'unit': '', 'price': 0.0, 'descClean': 'Visa', 'lineTotal': 30.0, 'productCode': ''}, {'qty': 14, 'desc': '009999 Date 14 / 06 / 19', 'unit': '', 'price': 0.0, 'descClean': 'Date / 06 / 19', 'lineTotal': 18.2, 'productCode': '009999'}]}"
1,SUCCESS: Result available,done,3,None,True,202,False,None,"{'establishment': 'Harrods', 'validatedEstablishment': False, 'date': '2019-06-19 13:24:00', 'total': 86.0, 'url': 'harrods.com', 'phoneNumber': '020 7730 1234', 'paymentMethod': '', 'address': 'LONDON S1 ot Road', 'validatedTotal': False, 'subTotal': 86.0, 'validatedSubTotal': True, 'cash': '0.000', 'change': '0.000', 'tax': 0.0, 'discount': 0.0, 'subTotalConfidence': 0.6, 'totalConfidence': 0.0, 'cashConfidence': 0.0, 'changeConfidence': 0.0, 'roundingConfidence': 0.0, 'otherData': [], 'lineItems': [{'qty': 0, 'desc': 'Rewards 0005708615 Saving FRAGRANCE', 'unit': '', 'price': 0.0, 'descClean': 'Rewards Saving FRAGRANCE f8.60', 'lineTotal': 86.0, 'productCode': '0005708615'}], 'summaryItems': [{'qty': 0, 'desc': 'Sub Total', 'unit': '', 'price': 0.0, 'descClean': 'Sub Total', 'lineTotal': 86.0, 'productCode': ''}, {'qty': 1, 'desc': 'TOTAL ( 1 item )', 'unit': '', 'price': 0.0, 'descClean': 'TOTAL ( item )', 'lineTotal': 77.4, 'productCode': ''}, {'qty': 0, 'desc': 'Chinese Mobile Tendered', 'unit': '', 'price': 0.0, 'descClean': 'Chinese Mobile Tendered', 'lineTotal': 77.4, 'productCode': ''}, {'qty': 0, 'desc': 'Rewards Points balance', 'unit': '', 'price': 0.0, 'descClean': 'Rewards Points balance', 'lineTotal': 194.8, 'productCode': ''}]}"


In [17]:
joined = df.join(parsedRawData)
data = joined.filter(['retailerName', 'result'])

In [18]:
# parse json result field and add columns to dataframe
parsedResult = data.result.apply(pd.Series)
joined = df.join(parsedResult)

# filter so that we get only features and labels to train the model
data = joined.filter(['establishment', 'retailerName'])
data.head(5)

,establishment,retailerName
0,Selfridges,Selfridges
1,Harrods,Harrods
2,Boots,Boots
3,Selfridges,Selfridges
4,John Lewis,John Lewis


In [19]:
ignore = data[data.establishment.apply(lambda x: not isinstance(x, str))]
ignore.head(2)

,establishment,retailerName
685,NaN,Boots
1326,NaN,Boots


In [20]:
ignore = data[data.retailerName.apply(lambda x: not isinstance(x, str))]
ignore.head(2)

,establishment,retailerName


In [21]:
data = data[data.establishment.apply(lambda x: isinstance(x, str))]
# data = data[data.retailerName.apply(lambda x: isinstance(x, str))]
data.head(10)

,establishment,retailerName
0,Selfridges,Selfridges
1,Harrods,Harrods
2,Boots,Boots
3,Selfridges,Selfridges
4,John Lewis,John Lewis
5,Selfridges,Selfridges
6,HOLLAND & BARRETT,Holland & Barrett
7,Selfridges,Selfridges
8,Selfridges,Selfridges
9,John Lewis,John Lewis


In [22]:
# get train and test dataset
train_size = int(len(data) * .8)
train_ocr = data['establishment'][:train_size]
train_tags = data['retailerName'][:train_size]
test_ocr = data['establishment'][train_size:]
test_tags = data['retailerName'][train_size:]

test_tags.size, train_size, len(data)

(1397, 5585, 6982)

In [23]:
train_ocr.head(10)

0           Selfridges
1              Harrods
2                Boots
3           Selfridges
4           John Lewis
5           Selfridges
6    HOLLAND & BARRETT
7           Selfridges
8           Selfridges
9           John Lewis
Name: establishment, dtype: object

In [24]:
# use bag of words model
max_words = 1000
tokenize = Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_ocr)

In [25]:
x_train = tokenize.texts_to_matrix(train_ocr)
x_test = tokenize.texts_to_matrix(test_ocr)

# x_train[0]

In [ ]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

y_test

In [ ]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)
y_test_cat

In [ ]:
# Inspect the dimensions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train_cat shape:', y_train_cat.shape)
print('y_test_cat shape:', y_test_cat.shape)

In [ ]:
batch_size = 32
epochs = 4

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(512, activation='relu', name='hidden_layer', input_shape=(max_words,)),
    layers.Dense(num_classes, activation='softmax', name='output')
])

In [ ]:
LR = 1e-5 # Keep it small when transfer learning
EPOCHS = 30
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss='categorical_crossentropy',
  metrics=['accuracy'])

In [ ]:
# train the model
history = model.fit(x_train, y_train_cat,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

In [ ]:
# evaluate the accuracy
score = model.evaluate(x_test, y_test_cat, batch_size=batch_size, verbose=1)
print('Test accuracy: {0:.2f}%'.format(score[1] * 100))

In [ ]:
text_labels = encoder.classes_ 
for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_ocr.iloc[i][:50], "...")
    print('correct:' + test_tags.iloc[i])
    print("predicted: " + predicted_label + "\n")

In [ ]:
encoder.classes_


In [ ]:
encoder.classes_
matrix = tokenize.texts_to_matrix(['HTK UK'])
prediction = model.predict(np.array([matrix[0]]))
predicted_label = text_labels[np.argmax(prediction)]
print("predict:{0}\nindex:{1} \nlabel:{2}".format(prediction, np.argmax(prediction), predicted_label))

In [ ]:
y_softmax = model.predict(x_test)

y_test_1d = []
y_pred_1d = []

for i in range(len(y_test_cat)):
    probs = y_test_cat[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_1d.append(one_hot_index)

for i in range(0, len(y_softmax)):
    probs = y_softmax[i]
    predicted_index = np.argmax(probs)
    y_pred_1d.append(predicted_index)

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Correct label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [ ]:
len(y_test_1d), len(y_pred_1d)

In [ ]:
cnf_matrix = confusion_matrix(y_test_1d, y_pred_1d)

In [ ]:
plt.figure(figsize=(24,20))
plot_confusion_matrix(cnf_matrix, classes=text_labels, title="Confusion matrix")
plt.show()